In [ ]:
%framework=tensorflow
%ps=1
%worker=1

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import math
import os
import sys
import tempfile
import time

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

flags = tf.app.flags
flags.DEFINE_string("data_dir", "/tmp/mnist-data",
                    "Directory for storing mnist data")
flags.DEFINE_boolean("download_only", False,
                     "Only perform downloading of data; Do not proceed to "
                     "session preparation, model definition or training")
flags.DEFINE_integer("task_index", None,
                     "Worker task index, should be >= 0. task_index=0 is "
                     "the master worker task the performs the variable "
                     "initialization ")
flags.DEFINE_integer("num_gpus", 1, "Total number of gpus for each machine."
                     "If you don't use GPU, please set it to '0'")
flags.DEFINE_integer("replicas_to_aggregate", None,
                     "Number of replicas to aggregate before parameter update"
                     "is applied (For sync_replicas mode only; default: "
                     "num_workers)")
flags.DEFINE_integer("hidden_units", 100,
                     "Number of units in the hidden layer of the NN")
flags.DEFINE_integer("train_steps", 20000,
                     "Number of (global) training steps to perform")
flags.DEFINE_integer("batch_size", 100, "Training batch size")
flags.DEFINE_float("learning_rate", 0.01, "Learning rate")
flags.DEFINE_boolean(
    "sync_replicas", False,
    "Use the sync_replicas (synchronized replicas) mode, "
    "wherein the parameter updates from workers are aggregated "
    "before applied to avoid stale gradients")
flags.DEFINE_boolean(
    "existing_servers", False, "Whether servers already exists. If True, "
    "will use the worker hosts via their GRPC URLs (one client process "
    "per worker host). Otherwise, will create an in-process TensorFlow "
    "server.")
flags.DEFINE_string("ps_hosts", "localhost:2222",
                    "Comma-separated list of hostname:port pairs")
flags.DEFINE_string("worker_hosts", "localhost:2223,localhost:2224",
                    "Comma-separated list of hostname:port pairs")
flags.DEFINE_string("job_name", None, "job name: worker or ps")

FLAGS = flags.FLAGS

IMAGE_PIXELS = 28

# Example:
#   cluster = {'ps': ['host1:2222', 'host2:2222'],
#              'worker': ['host3:2222', 'host4:2222', 'host5:2222']}
#   os.environ['TF_CONFIG'] = json.dumps(
#       {'cluster': cluster,
#        'task': {'type': 'worker', 'index': 1}})

def main(unused_argv):
  # Parse environment variable TF_CONFIG to get job_name and task_index

  # If not explicitly specified in the constructor and the TF_CONFIG
  # environment variable is present, load cluster_spec from TF_CONFIG.
  tf_config = json.loads(os.environ.get('TF_CONFIG') or '{}')
  task_config = tf_config.get('task', {})
  task_type = task_config.get('type')
  task_index = task_config.get('index')

  FLAGS.job_name = task_type
  FLAGS.task_index = task_index

  mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)
  if FLAGS.download_only:
    sys.exit(0)

  if FLAGS.job_name is None or FLAGS.job_name == "":
    raise ValueError("Must specify an explicit `job_name`")
  if FLAGS.task_index is None or FLAGS.task_index == "":
    raise ValueError("Must specify an explicit `task_index`")

  print("job name = %s" % FLAGS.job_name)
  print("task index = %d" % FLAGS.task_index)

  cluster_config = tf_config.get('cluster', {})
  ps_hosts = cluster_config.get('ps')
  worker_hosts = cluster_config.get('worker')

  ps_hosts_str = ','.join(ps_hosts)
  worker_hosts_str = ','.join(worker_hosts)

  FLAGS.ps_hosts = ps_hosts_str
  FLAGS.worker_hosts = worker_hosts_str

  # Construct the cluster and start the server
  ps_spec = FLAGS.ps_hosts.split(",")
  worker_spec = FLAGS.worker_hosts.split(",")

  # Get the number of workers.
  num_workers = len(worker_spec)

  cluster = tf.train.ClusterSpec({"ps": ps_spec, "worker": worker_spec})

  if not FLAGS.existing_servers:
    # Not using existing servers. Create an in-process server.
    server = tf.train.Server(
        cluster, job_name=FLAGS.job_name, task_index=FLAGS.task_index)
    if FLAGS.job_name == "ps":
      server.join()

  is_chief = (FLAGS.task_index == 0)
  if FLAGS.num_gpus > 0:
    # Avoid gpu allocation conflict: now allocate task_num -> #gpu
    # for each worker in the corresponding machine
    gpu = (FLAGS.task_index % FLAGS.num_gpus)
    worker_device = "/job:worker/task:%d/gpu:%d" % (FLAGS.task_index, gpu)
  elif FLAGS.num_gpus == 0:
    # Just allocate the CPU to worker server
    cpu = 0
    worker_device = "/job:worker/task:%d/cpu:%d" % (FLAGS.task_index, cpu)
  # The device setter will automatically place Variables ops on separate
  # parameter servers (ps). The non-Variable ops will be placed on the workers.
  # The ps use CPU and workers use corresponding GPU
  with tf.device(
      tf.train.replica_device_setter(
          worker_device=worker_device,
          ps_device="/job:ps/cpu:0",
          cluster=cluster)):
    global_step = tf.Variable(0, name="global_step", trainable=False)

    # Variables of the hidden layer
    hid_w = tf.Variable(
        tf.truncated_normal(
            [IMAGE_PIXELS * IMAGE_PIXELS, FLAGS.hidden_units],
            stddev=1.0 / IMAGE_PIXELS),
        name="hid_w")
    hid_b = tf.Variable(tf.zeros([FLAGS.hidden_units]), name="hid_b")

    # Variables of the softmax layer
    sm_w = tf.Variable(
        tf.truncated_normal(
            [FLAGS.hidden_units, 10],
            stddev=1.0 / math.sqrt(FLAGS.hidden_units)),
        name="sm_w")
    sm_b = tf.Variable(tf.zeros([10]), name="sm_b")

    # Ops: located on the worker specified with FLAGS.task_index
    x = tf.placeholder(tf.float32, [None, IMAGE_PIXELS * IMAGE_PIXELS])
    y_ = tf.placeholder(tf.float32, [None, 10])

    hid_lin = tf.nn.xw_plus_b(x, hid_w, hid_b)
    hid = tf.nn.relu(hid_lin)

    y = tf.nn.softmax(tf.nn.xw_plus_b(hid, sm_w, sm_b))
    cross_entropy = -tf.reduce_sum(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))

    opt = tf.train.AdamOptimizer(FLAGS.learning_rate)

    if FLAGS.sync_replicas:
      if FLAGS.replicas_to_aggregate is None:
        replicas_to_aggregate = num_workers
      else:
        replicas_to_aggregate = FLAGS.replicas_to_aggregate

      opt = tf.train.SyncReplicasOptimizer(
          opt,
          replicas_to_aggregate=replicas_to_aggregate,
          total_num_replicas=num_workers,
          name="mnist_sync_replicas")

    train_step = opt.minimize(cross_entropy, global_step=global_step)

    if FLAGS.sync_replicas:
      local_init_op = opt.local_step_init_op
      if is_chief:
        local_init_op = opt.chief_init_op

      ready_for_local_init_op = opt.ready_for_local_init_op

      # Initial token and chief queue runners required by the sync_replicas mode
      chief_queue_runner = opt.get_chief_queue_runner()
      sync_init_op = opt.get_init_tokens_op()

    init_op = tf.global_variables_initializer()
    train_dir = tempfile.mkdtemp()

    if FLAGS.sync_replicas:
      sv = tf.train.Supervisor(
          is_chief=is_chief,
          logdir=train_dir,
          init_op=init_op,
          local_init_op=local_init_op,
          ready_for_local_init_op=ready_for_local_init_op,
          recovery_wait_secs=1,
          global_step=global_step)
    else:
      sv = tf.train.Supervisor(
          is_chief=is_chief,
          logdir=train_dir,
          init_op=init_op,
          recovery_wait_secs=1,
          global_step=global_step)

    sess_config = tf.ConfigProto(
        allow_soft_placement=True,
        log_device_placement=False,
        device_filters=["/job:ps",
                        "/job:worker/task:%d" % FLAGS.task_index])

    # The chief worker (task_index==0) session will prepare the session,
    # while the remaining workers will wait for the preparation to complete.
    if is_chief:
      print("Worker %d: Initializing session..." % FLAGS.task_index)
    else:
      print("Worker %d: Waiting for session to be initialized..." %
            FLAGS.task_index)

    if FLAGS.existing_servers:
      server_grpc_url = "grpc://" + worker_spec[FLAGS.task_index]
      print("Using existing server at: %s" % server_grpc_url)

      sess = sv.prepare_or_wait_for_session(server_grpc_url, config=sess_config)
    else:
      sess = sv.prepare_or_wait_for_session(server.target, config=sess_config)

    print("Worker %d: Session initialization complete." % FLAGS.task_index)

    if FLAGS.sync_replicas and is_chief:
      # Chief worker will start the chief queue runner and call the init op.
      sess.run(sync_init_op)
      sv.start_queue_runners(sess, [chief_queue_runner])

    # Perform training
    time_begin = time.time()
    print("Training begins @ %f" % time_begin)

    local_step = 0
    while True:
      # Training feed
      batch_xs, batch_ys = mnist.train.next_batch(FLAGS.batch_size)
      train_feed = {x: batch_xs, y_: batch_ys}

      _, step = sess.run([train_step, global_step], feed_dict=train_feed)
      local_step += 1

      now = time.time()
      print("%f: Worker %d: training step %d done (global step: %d)" %
            (now, FLAGS.task_index, local_step, step))

      if step >= FLAGS.train_steps:
        break

    time_end = time.time()
    print("Training ends @ %f" % time_end)
    training_time = time_end - time_begin
    print("Training elapsed time: %f s" % training_time)

    # Validation feed
    val_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
    val_xent = sess.run(cross_entropy, feed_dict=val_feed)
    print("After %d training step(s), validation cross entropy = %g" %
          (FLAGS.train_steps, val_xent))


if __name__ == "__main__":
    tf.app.run()

[kubeflow] Building the Docker image...
[kubeflow] Image built successfully
[kubeflow] Getting tensorflow Job jupyter-kernel-pbhdg
[kubeflow] Wating for 1 PS and 1 workers
[kubeflow] There are 2 pods for the job.
[kubeflow][worker-0] Pod is pending...
[kubeflow][ps-0] Begin reading the log...
[kubeflow][ps-0]  -----> Running code...
[kubeflow][ps-0] WARNING:tensorflow:From /app/code.py:68: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
[kubeflow][ps-0] Instructions for updating:
[kubeflow][ps-0] Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
[kubeflow][ps-0] WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
[kubeflow][ps-0] Instructions for updating:
[kubeflow][ps-0] Ple

[kubeflow][worker-0] 1536376016.910831: Worker 0: training step 13 done (global step: 12)
[kubeflow][worker-0] 1536376016.914299: Worker 0: training step 14 done (global step: 13)
[kubeflow][worker-0] 1536376016.917221: Worker 0: training step 15 done (global step: 14)
[kubeflow][worker-0] 1536376016.921055: Worker 0: training step 16 done (global step: 15)
[kubeflow][worker-0] 1536376016.924331: Worker 0: training step 17 done (global step: 16)
[kubeflow][worker-0] 1536376016.927953: Worker 0: training step 18 done (global step: 17)
[kubeflow][worker-0] 1536376016.931152: Worker 0: training step 19 done (global step: 18)
[kubeflow][worker-0] 1536376016.935369: Worker 0: training step 20 done (global step: 19)
[kubeflow][worker-0] 1536376016.940414: Worker 0: training step 21 done (global step: 20)
[kubeflow][worker-0] 1536376016.943805: Worker 0: training step 22 done (global step: 21)
[kubeflow][worker-0] 1536376016.947057: Worker 0: training step 23 done (global step: 22)
[kubeflow]

[kubeflow][worker-0] 1536376017.179877: Worker 0: training step 110 done (global step: 109)
[kubeflow][worker-0] 1536376017.182392: Worker 0: training step 111 done (global step: 110)
[kubeflow][worker-0] 1536376017.185807: Worker 0: training step 112 done (global step: 111)
[kubeflow][worker-0] 1536376017.189157: Worker 0: training step 113 done (global step: 112)
[kubeflow][worker-0] 1536376017.194220: Worker 0: training step 114 done (global step: 113)
[kubeflow][worker-0] 1536376017.198141: Worker 0: training step 115 done (global step: 114)
[kubeflow][worker-0] 1536376017.202277: Worker 0: training step 116 done (global step: 115)
[kubeflow][worker-0] 1536376017.206599: Worker 0: training step 117 done (global step: 116)
[kubeflow][worker-0] 1536376017.211243: Worker 0: training step 118 done (global step: 117)
[kubeflow][worker-0] 1536376017.216091: Worker 0: training step 119 done (global step: 118)
[kubeflow][worker-0] 1536376017.222203: Worker 0: training step 120 done (global

[kubeflow][worker-0] 1536376017.467422: Worker 0: training step 205 done (global step: 204)
[kubeflow][worker-0] 1536376017.469480: Worker 0: training step 206 done (global step: 205)
[kubeflow][worker-0] 1536376017.471505: Worker 0: training step 207 done (global step: 206)
[kubeflow][worker-0] 1536376017.474027: Worker 0: training step 208 done (global step: 207)
[kubeflow][worker-0] 1536376017.476777: Worker 0: training step 209 done (global step: 208)
[kubeflow][worker-0] 1536376017.478981: Worker 0: training step 210 done (global step: 209)
[kubeflow][worker-0] 1536376017.481272: Worker 0: training step 211 done (global step: 210)
[kubeflow][worker-0] 1536376017.483627: Worker 0: training step 212 done (global step: 211)
[kubeflow][worker-0] 1536376017.486142: Worker 0: training step 213 done (global step: 212)
[kubeflow][worker-0] 1536376017.488251: Worker 0: training step 214 done (global step: 213)
[kubeflow][worker-0] 1536376017.490341: Worker 0: training step 215 done (global

[kubeflow][worker-0] 1536376017.774184: Worker 0: training step 315 done (global step: 314)
[kubeflow][worker-0] 1536376017.776616: Worker 0: training step 316 done (global step: 315)
[kubeflow][worker-0] 1536376017.778824: Worker 0: training step 317 done (global step: 316)
[kubeflow][worker-0] 1536376017.780998: Worker 0: training step 318 done (global step: 317)
[kubeflow][worker-0] 1536376017.783352: Worker 0: training step 319 done (global step: 318)
[kubeflow][worker-0] 1536376017.786033: Worker 0: training step 320 done (global step: 319)
[kubeflow][worker-0] 1536376017.788765: Worker 0: training step 321 done (global step: 320)
[kubeflow][worker-0] 1536376017.791444: Worker 0: training step 322 done (global step: 321)
[kubeflow][worker-0] 1536376017.793779: Worker 0: training step 323 done (global step: 322)
[kubeflow][worker-0] 1536376017.796296: Worker 0: training step 324 done (global step: 323)
[kubeflow][worker-0] 1536376017.798506: Worker 0: training step 325 done (global

...

[kubeflow][worker-0] 1536376074.457048: Worker 0: training step 19535 done (global step: 19534)
[kubeflow][worker-0] 1536376074.460115: Worker 0: training step 19536 done (global step: 19535)
[kubeflow][worker-0] 1536376074.463456: Worker 0: training step 19537 done (global step: 19536)
[kubeflow][worker-0] 1536376074.466487: Worker 0: training step 19538 done (global step: 19537)
[kubeflow][worker-0] 1536376074.470455: Worker 0: training step 19539 done (global step: 19538)
[kubeflow][worker-0] 1536376074.474164: Worker 0: training step 19540 done (global step: 19539)
[kubeflow][worker-0] 1536376074.477209: Worker 0: training step 19541 done (global step: 19540)
[kubeflow][worker-0] 1536376074.480438: Worker 0: training step 19542 done (global step: 19541)
[kubeflow][worker-0] 1536376074.484953: Worker 0: training step 19543 done (global step: 19542)
[kubeflow][worker-0] 1536376074.491651: Worker 0: training step 19544 done (global step: 19543)
[kubeflow][worker-0] 1536376074.495003: 

[kubeflow][worker-0] 1536376074.831449: Worker 0: training step 19645 done (global step: 19644)
[kubeflow][worker-0] 1536376074.834504: Worker 0: training step 19646 done (global step: 19645)
[kubeflow][worker-0] 1536376074.838582: Worker 0: training step 19647 done (global step: 19646)
[kubeflow][worker-0] 1536376074.842562: Worker 0: training step 19648 done (global step: 19647)
[kubeflow][worker-0] 1536376074.845901: Worker 0: training step 19649 done (global step: 19648)
[kubeflow][worker-0] 1536376074.850321: Worker 0: training step 19650 done (global step: 19649)
[kubeflow][worker-0] 1536376074.855541: Worker 0: training step 19651 done (global step: 19650)
[kubeflow][worker-0] 1536376074.859667: Worker 0: training step 19652 done (global step: 19651)
[kubeflow][worker-0] 1536376074.862998: Worker 0: training step 19653 done (global step: 19652)
[kubeflow][worker-0] 1536376074.866014: Worker 0: training step 19654 done (global step: 19653)
[kubeflow][worker-0] 1536376074.869708: 

[kubeflow][worker-0] 1536376075.119435: Worker 0: training step 19737 done (global step: 19736)
[kubeflow][worker-0] 1536376075.121893: Worker 0: training step 19738 done (global step: 19737)
[kubeflow][worker-0] 1536376075.124418: Worker 0: training step 19739 done (global step: 19738)
[kubeflow][worker-0] 1536376075.127388: Worker 0: training step 19740 done (global step: 19739)
[kubeflow][worker-0] 1536376075.130638: Worker 0: training step 19741 done (global step: 19740)
[kubeflow][worker-0] 1536376075.133641: Worker 0: training step 19742 done (global step: 19741)
[kubeflow][worker-0] 1536376075.136304: Worker 0: training step 19743 done (global step: 19742)
[kubeflow][worker-0] 1536376075.139060: Worker 0: training step 19744 done (global step: 19743)
[kubeflow][worker-0] 1536376075.141486: Worker 0: training step 19745 done (global step: 19744)
[kubeflow][worker-0] 1536376075.144321: Worker 0: training step 19746 done (global step: 19745)
[kubeflow][worker-0] 1536376075.147382: 

[kubeflow][worker-0] 1536376075.514077: Worker 0: training step 19847 done (global step: 19846)
[kubeflow][worker-0] 1536376075.519885: Worker 0: training step 19848 done (global step: 19847)
[kubeflow][worker-0] 1536376075.523325: Worker 0: training step 19849 done (global step: 19848)
[kubeflow][worker-0] 1536376075.525926: Worker 0: training step 19850 done (global step: 19849)
[kubeflow][worker-0] 1536376075.528279: Worker 0: training step 19851 done (global step: 19850)
[kubeflow][worker-0] 1536376075.530817: Worker 0: training step 19852 done (global step: 19851)
[kubeflow][worker-0] 1536376075.534032: Worker 0: training step 19853 done (global step: 19852)
[kubeflow][worker-0] 1536376075.537460: Worker 0: training step 19854 done (global step: 19853)
[kubeflow][worker-0] 1536376075.540559: Worker 0: training step 19855 done (global step: 19854)
[kubeflow][worker-0] 1536376075.543448: Worker 0: training step 19856 done (global step: 19855)
[kubeflow][worker-0] 1536376075.546144: 

[kubeflow][worker-0] 1536376075.799637: Worker 0: training step 19933 done (global step: 19932)
[kubeflow][worker-0] 1536376075.802118: Worker 0: training step 19934 done (global step: 19933)
[kubeflow][worker-0] 1536376075.804493: Worker 0: training step 19935 done (global step: 19934)
[kubeflow][worker-0] 1536376075.807569: Worker 0: training step 19936 done (global step: 19935)
[kubeflow][worker-0] 1536376075.810539: Worker 0: training step 19937 done (global step: 19936)
[kubeflow][worker-0] 1536376075.813831: Worker 0: training step 19938 done (global step: 19937)
[kubeflow][worker-0] 1536376075.817495: Worker 0: training step 19939 done (global step: 19938)
[kubeflow][worker-0] 1536376075.820138: Worker 0: training step 19940 done (global step: 19939)
[kubeflow][worker-0] 1536376075.822969: Worker 0: training step 19941 done (global step: 19940)
[kubeflow][worker-0] 1536376075.826462: Worker 0: training step 19942 done (global step: 19941)
[kubeflow][worker-0] 1536376075.829704: 